## 10장. 딥러닝 모델 설계하기

### 2. 입력층, 은닉층, 출력층

```python
model = Sequential()
model.add(Dense(30, input_dim=16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
```

- 우선 `Sequential()`으로 딥러닝의 층을 구현합니다.
- 그리고 `model.add()`를 사용해 새로운 층을 추가합니다.
    - 위의 코드에는 2개의 층을 가진 모델을 만들고 있습니다.
- 각 층은 Dense 함수를 통해 구체적인 구조를 결정합니다.
    - 첫 번째 인자에는 몇 개의 노드를 만들 것인지를 작성합니다. 
    - 두 번째 인자 `input_dim`에는 입력 데이터(x)로부터 몇 개의 값을 가져올 지를 정합니다. 즉 첫 번째 Dense 에서 입력층과 은닉층을 결정합니다.
- 다음 두 번째 Dense 는 마지막 출력층입니다.
    - 출력 값은 1개이므로 첫 인자는 1입니다.
    - 입력을 받은 값을 시그모이드 활성화 함수를 거쳐 최종 출력 값으로 보여줍니다.

### 3. 모델 컴파일

위의 2장에서 만든 모델을 효과적으로 구현하기 위한 환경 설정을 합니다.

```python
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
```

- `loss`에서 오차 함수를 결정합니다. 여기서는 생존, 사망 둘 중 하나를 예측해야 하니까 교차 엔트로피 오차 함수를 적용했습니다.
- 두 번째로 옵티마이저를 선택합니다.
- `metrics` 함수는 모델이 컴파일 될 때 모델을 수행한 결과를 나타내도록 하는 것입니다.
    - "accuracy" 는 힉습셋에 대해 정확도에 기반하여 결과를 출력하라는 뜻입니다.
    - 그 외에도 학습셋의 손실값을 나타내는 "loss", 테스트셋의 정확도를 나타내는 "val_acc", 테스트셋의 손실값을 나타내는 "val_loss" 등이 있습니다.

### 4. 모델 실행하기

현재 470명의 환자(샘플), 16개의 정보(속성), 생존 여부(클래스)가 데이터가 있습니다.

```python
history = model.fit(X, y, epochs=5, batch_size=16)
```

- 학습 프로세스가 모든 샘플에 대해 한 번 실행하는 것을 `epoch=1`이라 합니다. 여기서는 각 샘플을 5번 재사용하라고 설정했습니다.
- `batch_size`는 샘플을 한 번에 몇 개씩 처리할 것인지를 정의합니다.
    - 이 값이 너무 크면 학습 속도가 느려지고, 너무 작으면 각 실행 값에 편차가 생겨서 전체 결과가 불안정해질 수 있습니다.
    - 따라서 컴퓨터 메모리가 감당할 만큼의 `batch_size`를 설정해야 합니다.

## 11장. 데이터 다루기

한쪽으로 치우치지 않고, 불필요한 정보가 대량으로 포함되어 있지 않으며, 왜곡되지 않은 데이터가 얼마나 있느냐가 중요합니다. 따라서 우선 데이터를 많이 모았다면, 그것을 머신러닝과 딥러닝에 사용할 수 있게 정제할 필요가 있습니다.

### 3. 판다스를 활용한 데이터 조사

```python
df = pd.read_csv('./data/pima-indians-diabetes3.csv')
```

- csv 파일을 가져와 데이터 프레임으로 저장합니다.
- `df.head(5)` 함수로 첫 다섯 행을 확인할 수 있습니다.
- 특정 열의 정보를 확인하려면 `df["컬럼명"]`을, 컬럼의 값을 알려면 `df["컬럼명"].value_counts()`를 사용합니다.
- `df.describe()`로 각 항목의 개수, 평균, 표준편차 등을 구할 수 있습니다.
- `df.corr()`으로 각 항목의 상관관계를 확인합니다.

### 4. 중요한 데이터 추출하기

`df.corr()`을 히트맵으로 그려 "plasma"와 "BMI" 항목이 "diabetes"하고 상관관계가 높은 것을 알 수 있습니다. 그것을 히스토그램으로 그려 분포를 확인할 수 있습니다.

```python
# "plasma"와 "diabetes"에 대한 히스토그램
plt.hist(x=[df.plasma[df.diables==0], df.plasma[df.diabetes==1]], bins=30, histtype='barstacked', label=['normal','diabetes'])
plt.legend()
```

이처럼 결과에 미치는 영향이 큰 항목을 발견하는 것이 데이터 전처리 과정 중 하나입니다. (그 밖에도 데이터에 빠진 값을 평균이나 중앙값으로 대치하거나, 이상치를 제거하는 작업도 데이터 전처리라 할 수 있습니다.)

### 5. 피마 인디언의 당뇨병 예측 실행

```python
df = pd.read_csv('./data/pima-indians-diabetes3.csv')
X = df.iloc[:, 0:8]
y = df.iloc[:, 8]

model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu', name='Dense_1'))
model.add(Dense(8, activation='relu', name='Dense_2'))
model.add(Dense(1, activation='sigmoid', name='Dense_3'))
model.summary()
```

- `model.summary()`로 각 층의 연결을 확인할 수 있습니다.

여기서는 은닉층을 2개 그리고 노드의 수를 12개와 8개로 임의로 지정했는데, 실제 프로젝트에서는 여러 번 반복하여 최적의 은닉층과 노드 개수를 찾아야 합니다.

